https://mp.weixin.qq.com/s/eo7MjFgs7pjbtRZ3R-DcEQm

In [1]:
import collections
import os 
import shutil
import tqdm
import numpy as np
import PIL.Image
import torch
import torchvision

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

#### 1.基础配置

In [3]:
# 检测pytorch版本
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torch.cuda.get_device_name(0))

1.2.0+cu92
9.2
7301
GeForce RTX 2070


In [4]:
# 更新pytorch
# conda update pytorch torchvision -c pytorch

In [5]:
# 固定随机种子
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)

In [6]:
# 指定程序运行在特定GPU卡上
# CUDA_VISIBLE_DEVICES=0,1 python train.py
# 或在代码中指定
# os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

In [7]:
# 判断是否有CUDA支持
print(torch.cuda.is_available())

True


In [8]:
# 设置为cuDNN benchmark模式
# benchmark模式会提升计算速度，但是由于计算机中有随机性，每次网络前馈结果略有差异
# torch.backends.cudnn.benchmark = True
# 如果想避免这种结果波动，设置
# torch.backends.cudnn.deterministic = True

In [10]:
# 清楚GPU存储
# 有时Control-C中止运行后GPU存储没有及时释放，需要手动清空，在pytorch内部可以
# torch.cuda.empty_cache()
# 或在命令行可以先使用ps找到程序的PID，再使用kill结束该进程
# ps aux | grep python
# kill -9 [pid]
# 或者直接重置没有被清空的GPU
# nvidia-smi --gpu-reset -i [gpu_id]

#### 张量处理

In [ ]:
# 张量基本信息
tensor.type()   # Data type
tensor.size()   # Shape of the tensor. It is a subclass of Python tuple
tensor.dim()    # Number of dimensions.

In [ ]:
# 数据类型转换
# Set default tensor type. Float in PyTorch is much faster than double.
torch.set_default_dtype(torch.FloatType)
# type convertions
tensor = tensor.cuda()
tensor = tensor.cpu()
tensor = tensor.float()
tensor = tensor.long()

In [ ]:
# torch.Tensor与np.ndarray转换
# torch.Tensor -> np.ndarray
ndarray = tensor.cpu().numpy()
# np.ndarray -> torch.Tensor
tensor = torch.from_numpy(ndarray).float()
tensor = torch.from_numpy(ndarray.copy()).float() # If ndarray has negative stride

In [ ]:
# torch.Tensor与PIL.Image转换
# pytorch中的张量默认采用N×C×H×W的顺序，并且数据范围在[0,1]需要进行转置和规范化
# torch.Tensor -> PIL.Image
image = PIL.Image.fromarray(torch.clamp(tensor * 255, min=0, max=255
    ).byte().permute(1, 2, 0).cpu().numpy())
image = torchvision.transforms.functional.to_pil_image(tensor)  # Equivalently way

# PIL.Image -> torch.Tensor.
tensor = torch.from_numpy(np.asarray(PIL.Image.open(path))
    ).permute(2, 0, 1).float() / 255
tensor = torchvision.transforms.functional.to_tensor(PIL.Image.open(path))  # Equivalently way

In [ ]:
# np.ndarray与PIL.Image转换
# np.ndarray -> PIL.Image
image = PIL.Image.fromarray(ndarray.astype(np.uint8))
# PIL.Image -> np.ndarray
ndarray = np.asarray(PIL.Image.open(path))

In [ ]:
# 从只包含一个元素的张量中提取值，这在训练时统计loss的变化过程中特别有用，否则这将累积计算图，使GPU存储占用量越来越大
value = tensor.item()

In [ ]:
# 张量变形，常常需要用于将卷积层特征输入全连接层的情形，相比torch.view,torch.reshape可以自动处理输入张量不连续的情况
tensor = torch.reshape(tensor,shape)

In [ ]:
# 打乱顺序
tensor = tensor[torch.randperm(tensor.size(0))] # Shuffle the first dimension

In [ ]:
# 水平翻转，pytorch不支持tensor[::-1]这样的负步长操作，水平翻转可以用张量索引实现
tensor = tensor[:,:,:,torch.arange(tensor.size(3) -1,-1,-1).long()]

In [ ]:
# 复制张量，有三种复制的方式，对应不同的需求
# Operation                 |  New/Shared memory | Still in computation graph |
tensor.clone()            # |        New         |          Yes               |
tensor.detach()           # |      Shared        |          No                |
tensor.detach.clone()()   # |        New         |          No  

In [ ]:
# 拼接张量，注意torch.cat和torch.stack的区别在于torch.cat沿着给定的维度拼接，而torch.stack会新增一维，例如当参数是3个10×5的张量，torch.cat的结果是30×5的张量，而torch.stack的结果是3×10×5的张量。
tensor = torch.cat(list_of_tensors, dim=0)
tensor = torch.stack(list_of_tensors, dim=0)

In [ ]:
# 将整数标记转换成独热（one-hot）编码,pytorch中的标记默认从0开始
N  = tensor.size(0)
one_hot = torch.zeros(N,num_classes).long()
one_hot.scatter_(dim=1,index=torch.unsqueeze(tensor,dim=1),src=torch.ones(N.num_classes).long())

In [ ]:
# 得到非零/零元素
torch.nonzero(tensor)
torch.nonzero(tensor == 0)
torch.nonzero(tensor).size(0)
torch.nonzero(tensor == 0).size(0)

In [ ]:
# 判断两个张量相等
torch.allclose(tensor1,tensor2) # float tensor
torch.equal(tensor1,tensor2) # int tensor

In [ ]:
# 张量扩展
# # Expand tensor of shape 64*512 to shape 64*512*7*7.
torch.reshape(tensor,(64,512,1,1)).expand(64, 512, 7, 7)

In [ ]:
# 矩阵乘法
# Matrix multiplication: (m*n) * (n*p) -> (m*p).
result = torch.mm(tensor1, tensor2)

# Batch matrix multiplication: (b*m*n) * (b*n*p) -> (b*m*p).
result = torch.bmm(tensor1, tensor2)

# Element-wise multiplication.
result = tensor1 * tensor2

In [ ]:
# 计算两组数据之间的欧式距离
dist = torch.sqrt(torch.sum((X1[:,None,:] - X2) ** 2,dim=2))

#### 模型定义

In [ ]:
# 卷积层 最常用的卷积层配置是
conv = torch.nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=True)
conv = torch.nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=True)

In [ ]:
# GAP(Global average pooling)层
gap = torch.nn.AdaptiveAvgPool2d(output_size=1)

In [ ]:
# 双线性汇合（bilinear pooling）
X = torch.reshape(N, D, H * W)                        # Assume X has shape N*D*H*W
X = torch.bmm(X, torch.transpose(X, 1, 2)) / (H * W)  # Bilinear pooling
assert X.size() == (N, D, D)
X = torch.reshape(X, (N, D * D))
X = torch.sign(X) * torch.sqrt(torch.abs(X) + 1e-5)   # Signed-sqrt normalization
X = torch.nn.functional.normalize(X)                  # L2 normalization

In [ ]:
# 多卡同步BN(Batch normalization)
sync_bn = torch.nn.SyncBatchNorm(num_features,eps=1e-05,momentum=0.1,affine=True,track_running_stats=True)
# 将已有网络的所有BN层改为同步BN层

In [ ]:
# 类型BN滑动平均，如果要实现类型BN滑动平均的操作，在forward函数中要使用原地（inplace）操作给滑动平均赋值